Kancharla Naga Jahnavi

assignment 09

In [2]:
#importing required modules
import os
import sys
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
from patsy import dmatrices
from sklearn.metrics import confusion_matrix
import sklearn
from sklearn import datasets

In [3]:
## Setting  default figure size to be larger
## this may only work in matplotlib 2.0+!
matplotlib.rcParams['figure.figsize'] = [10.0,6.0]
## Enabling  multiple outputs from jupyter cells
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [5]:
# Working Directory
import os
print("My working directory:\n" + os.getcwd())
# Set Working Directory
os.chdir(".")
print("My new working directory:\n" + os.getcwd())

My working directory:
/content
My new working directory:
/content


In [6]:
#importing data
df_patient = \
 pd.read_csv('/content/PatientAnalyticFile.csv')
df_patient

,PatientID,DateOfBirth,Gender,Race,Myocardial_infarction,Congestive_heart_failure,Peripheral_vascular_disease,Stroke,Dementia,Pulmonary,...,Metastatic_solid_tumour,HIV,Obesity,Depression,Hypertension,Drugs,Alcohol,First_Appointment_Date,Last_Appointment_Date,DateOfDeath
0,1,1962-02-27,female,hispanic,0,0,0,0,0,0,...,0,0,0,0,0,0,0,2013-04-27,2018-06-01,NaN
1,2,1959-08-18,male,white,0,0,0,0,0,0,...,0,0,0,0,1,0,0,2005-11-30,2008-11-02,2008-11-02
2,3,1946-02-15,female,white,0,0,0,0,0,0,...,0,1,0,0,1,0,0,2011-11-05,2015-11-13,NaN
3,4,1979-07-27,female,white,0,0,0,0,0,1,...,0,0,0,0,0,0,0,2010-03-01,2016-01-17,2016-01-17
4,5,1983-02-19,female,hispanic,0,0,0,0,0,0,...,0,0,0,0,1,0,0,2006-09-22,2018-06-01,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,19996,1997-12-19,female,other,0,0,0,0,0,0,...,0,0,0,0,0,0,0,2008-06-14,2018-06-01,NaN
19996,19997,1984-03-31,female,white,0,0,0,0,0,0,...,0,1,0,0,1,0,0,2007-04-24,2018-06-01,NaN
19997,19998,1993-07-04,female,white,0,0,0,0,0,0,...,0,0,1,0,1,0,0,2010-10-16,2018-06-01,NaN
19998,19999,1984-04-17,male,other,0,0,0,0,0,0,...,0,0,0,0,1,0,0,2015-01-04,2018-06-01,NaN


In [7]:
#creating mortality variable
df_patient['mortality'] = \
    np.where(df_patient['DateOfDeath'].isnull(),
             0,1)
# Examine
df_patient['mortality']

,mortality
0,0
1,1
2,0
3,1
4,0
...,...
19995,0
19996,0
19997,0
19998,0


In [8]:
#describing the mortality variable
df_patient['mortality'].describe()

,mortality
count,20000.000000
mean,0.354700
std,0.478434
min,0.000000
25%,0.000000
50%,0.000000
75%,1.000000
max,1.000000


In [9]:
#displaying the datatypes of the data file
df_patient.dtypes

,0
PatientID,int64
DateOfBirth,object
Gender,object
Race,object
Myocardial_infarction,int64
Congestive_heart_failure,int64
Peripheral_vascular_disease,int64
Stroke,int64
Dementia,int64
Pulmonary,int64


In [10]:
# Convert dateofBirth to date
df_patient['DateOfBirth'] = \
    pd.to_datetime(df_patient['DateOfBirth'])
# Calculate age in years as of 2015-01-01
df_patient['Age_years'] = \
    ((pd.to_datetime('2015-01-01') - df_patient['DateOfBirth']).dt.days/365.25)
df_patient['Age_years'].describe()

,Age_years
count,20000.000000
mean,47.247474
std,18.145086
min,15.753593
25%,31.733744
50%,47.099247
75%,62.924025
max,78.743326


In [11]:
#displaying columns of the data file
df_patient.columns

Index(['PatientID', 'DateOfBirth', 'Gender', 'Race', 'Myocardial_infarction',
       'Congestive_heart_failure', 'Peripheral_vascular_disease', 'Stroke',
       'Dementia', 'Pulmonary', 'Rheumatic', 'Peptic_ulcer_disease',
       'LiverMild', 'Diabetes_without_complications',
       'Diabetes_with_complications', 'Paralysis', 'Renal', 'Cancer',
       'LiverSevere', 'Metastatic_solid_tumour', 'HIV', 'Obesity',
       'Depression', 'Hypertension', 'Drugs', 'Alcohol',
       'First_Appointment_Date', 'Last_Appointment_Date', 'DateOfDeath',
       'mortality', 'Age_years'],
      dtype='object')

In [12]:
#creating a formula for all the variables in the model
vars_remove = ['PatientID','First_Appointment_Date','DateOfBirth',
               'Last_Appointment_Date','DateOfDeath','mortality']
vars_left = set(df_patient.columns) - set(vars_remove)
formula = "mortality ~ " + " + ".join(vars_left)
formula

'mortality ~ LiverSevere + Renal + Paralysis + Metastatic_solid_tumour + Age_years + Pulmonary + Drugs + Race + Depression + Diabetes_without_complications + Myocardial_infarction + Diabetes_with_complications + HIV + LiverMild + Gender + Peptic_ulcer_disease + Stroke + Dementia + Peripheral_vascular_disease + Rheumatic + Alcohol + Obesity + Congestive_heart_failure + Hypertension + Cancer'

In [13]:
#only using subset of the data
df_patient_sub = \
    df_patient.sample(frac=0.1,
                     random_state=32)
## use Patsy to create model matrices
Y,X = dmatrices(formula,
                df_patient_sub)

In [14]:
Y

DesignMatrix with shape (2000, 1)
  mortality
          0
          0
          1
          1
          0
          0
          1
          1
          0
          0
          1
          0
          1
          0
          1
          0
          1
          0
          0
          1
          0
          1
          0
          0
          0
          0
          1
          1
          0
          0
  [1970 rows omitted]
  Terms:
    'mortality' (column 0)
  (to view full data, use np.asarray(this_obj))

In [15]:
X

DesignMatrix with shape (2000, 28)
  Columns:
    ['Intercept',
     'Race[T.hispanic]',
     'Race[T.other]',
     'Race[T.white]',
     'Gender[T.male]',
     'LiverSevere',
     'Renal',
     'Paralysis',
     'Metastatic_solid_tumour',
     'Age_years',
     'Pulmonary',
     'Drugs',
     'Depression',
     'Diabetes_without_complications',
     'Myocardial_infarction',
     'Diabetes_with_complications',
     'HIV',
     'LiverMild',
     'Peptic_ulcer_disease',
     'Stroke',
     'Dementia',
     'Peripheral_vascular_disease',
     'Rheumatic',
     'Alcohol',
     'Obesity',
     'Congestive_heart_failure',
     'Hypertension',
     'Cancer']
  Terms:
    'Intercept' (column 0)
    'Race' (columns 1:4)
    'Gender' (column 4)
    'LiverSevere' (column 5)
    'Renal' (column 6)
    'Paralysis' (column 7)
    'Metastatic_solid_tumour' (column 8)
    'Age_years' (column 9)
    'Pulmonary' (column 10)
    'Drugs' (column 11)
    'Depression' (column 12)
    'Diabetes_without_compl

In [16]:
#SPLITTING INTO TESTING AND TRAINING SAMPLES
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = \
    train_test_split(X,
                     np.ravel(Y), # prevents dimensionality error later!
                     test_size=0.25,
                     random_state=42)

First model: logistic regression

In [17]:
from sklearn import linear_model
## Define model parameters
## can implement penalties, but check docs for appropriate solver
clf = linear_model.LogisticRegression(fit_intercept=True, # already have the intercept
                                      solver='liblinear') # could change to lbfgs!
## fit model using data with .fit
clf.fit(X_train,y_train)

LogisticRegression(solver='liblinear')

In [18]:
## Making predictions on training dataset
## training error?
clf.predict(X_train)

array([0., 0., 0., ..., 0., 0., 0.])

In [19]:
## can also predict probabilities
clf.predict_proba(X_train)

array([[0.71461109, 0.28538891],
       [0.72980987, 0.27019013],
       [0.91840587, 0.08159413],
       ...,
       [0.91934317, 0.08065683],
       [0.93820382, 0.06179618],
       [0.7282174 , 0.2717826 ]])

Model summaries

In [20]:
clf.coef_
clf.coef_.shape

array([[-1.86246831, -0.11595603, -0.18726503, -0.0453132 , -0.10167484,
         0.71968793,  0.27342734, -0.01891693, -0.29372891,  0.05813761,
         0.33183451,  0.3111227 ,  0.56220816, -0.26651427,  0.5144615 ,
        -0.63768787,  0.09894361,  0.16254145, -0.34338787,  0.10084582,
        -0.01869835,  0.87534657,  0.40896697,  1.03493317,  0.25592985,
         0.40421821, -0.02121857,  0.42038745]])

(1, 28)

Assessing the accuracy

In [21]:
#mean accuracy
clf.score(X_train,y_train)

0.7333333333333333

In [22]:
#confusion matrix
confusion_matrix(y_train,
                 clf.predict(X_train))

array([[826, 144],
       [256, 274]])

In [23]:
#classification matrix
print(sklearn.metrics.classification_report(y_train,
                                            clf.predict(X_train)))

              precision    recall  f1-score   support

         0.0       0.76      0.85      0.81       970
         1.0       0.66      0.52      0.58       530

    accuracy                           0.73      1500
   macro avg       0.71      0.68      0.69      1500
weighted avg       0.73      0.73      0.72      1500



In [24]:
#accuracy
sklearn.metrics.accuracy_score(y_train,
                               clf.predict(X_train))

0.7333333333333333

In [25]:
#kappa
sklearn.metrics.cohen_kappa_score(y_train,
                                  clf.predict(X_train))

np.float64(0.3870796387856005)

In [26]:
#creating a dictionary to store the results
result_scores = {}
## Score the Model on Training and Testing Set
result_scores['Logistic'] = \
            (sklearn.metrics.accuracy_score(y_train,clf.predict(X_train)),
             sklearn.metrics.accuracy_score(y_test,clf.predict(X_test)))

In [27]:
## Creating Function to Print Results
def get_results(x1):
    print("\n{0:20}   {1:4}    {2:4}".format('Model','Train','Test'))
    print('-------------------------------------------')
    for i in x1.keys():
        print("{0:20}   {1:<6.4}   {2:<6.4}".format(i,x1[i][0],x1[i][1]))

In [28]:
get_results(result_scores)


Model                  Train    Test
-------------------------------------------
Logistic               0.7333   0.718 


In [29]:
#null model
1 - y_train.mean()

array(0.64666667)

In [30]:
from sklearn.dummy import DummyClassifier
clf = DummyClassifier(strategy='most_frequent',
                      random_state=0)
clf.fit(X_train, y_train)
clf.score(X_train, y_train)

DummyClassifier(random_state=0, strategy='most_frequent')

0.6466666666666666

In [31]:
## Scoring the Model on Training and Testing Set
result_scores['Null'] = \
            (sklearn.metrics.accuracy_score(y_train,clf.predict(X_train)),
             sklearn.metrics.accuracy_score(y_test,clf.predict(X_test)))
get_results(result_scores)


Model                  Train    Test
-------------------------------------------
Logistic               0.7333   0.718 
Null                   0.6467   0.608 


In [32]:
#Logistic regression with L1 penalty
clf = linear_model.LogisticRegression(penalty='l1',
                                      C=1, solver = 'liblinear') # specify penalty
clf.fit(X_train,y_train)
## get confusion matrix
confusion_matrix(y_train,clf.predict(X_train))

LogisticRegression(C=1, penalty='l1', solver='liblinear')

array([[820, 150],
       [252, 278]])

In [33]:
#accuracy
sklearn.metrics.accuracy_score(y_train,clf.predict(X_train))

0.732

In [34]:
## Getting  kappa
sklearn.metrics.cohen_kappa_score(y_train,clf.predict(X_train))

np.float64(0.3867713460521498)

In [35]:
### getting  classification metrics
print(sklearn.metrics.classification_report(y_train,
                                            clf.predict(X_train)))

              precision    recall  f1-score   support

         0.0       0.76      0.85      0.80       970
         1.0       0.65      0.52      0.58       530

    accuracy                           0.73      1500
   macro avg       0.71      0.68      0.69      1500
weighted avg       0.72      0.73      0.72      1500



In [36]:
## Scoring the Model on Training and Testing Set
result_scores['Logistic_L1_C_1'] = \
            (sklearn.metrics.accuracy_score(y_train,clf.predict(X_train)),
             sklearn.metrics.accuracy_score(y_test,clf.predict(X_test)))
get_results(result_scores)


Model                  Train    Test
-------------------------------------------
Logistic               0.7333   0.718 
Null                   0.6467   0.608 
Logistic_L1_C_1        0.732    0.716 


In [37]:
## Logistic Regression with l1 penalty
## Specify penalty directly as C = 0.1
clf = linear_model.LogisticRegression(penalty='l1',
                                      C=0.1, solver = 'liblinear') # specify penalty
clf.fit(X_train,y_train)
## Score the Model on Training and Testing Set
result_scores['Logistic_L1_C_01'] = \
            (sklearn.metrics.accuracy_score(y_train,clf.predict(X_train)),
             sklearn.metrics.accuracy_score(y_test,clf.predict(X_test)))
get_results(result_scores)

LogisticRegression(C=0.1, penalty='l1', solver='liblinear')


Model                  Train    Test
-------------------------------------------
Logistic               0.7333   0.718 
Null                   0.6467   0.608 
Logistic_L1_C_1        0.732    0.716 
Logistic_L1_C_01       0.726    0.706 


In [38]:
## Logistic Regression with l1 penalty
## Specify penalty directly as C = 0.1
clf = linear_model.LogisticRegression(penalty='l1',
                                      C=10, solver = 'liblinear') # specify penalty
clf.fit(X_train,y_train)
## Score the Model on Training and Testing Set
result_scores['Logistic_L1_C_10'] = \
            (sklearn.metrics.accuracy_score(y_train,clf.predict(X_train)),
             sklearn.metrics.accuracy_score(y_test,clf.predict(X_test)))
get_results(result_scores)

LogisticRegression(C=10, penalty='l1', solver='liblinear')


Model                  Train    Test
-------------------------------------------
Logistic               0.7333   0.718 
Null                   0.6467   0.608 
Logistic_L1_C_1        0.732    0.716 
Logistic_L1_C_01       0.726    0.706 
Logistic_L1_C_10       0.7347   0.718 


#selecting parameters via cross validation

In [39]:
## Select the alpha through cross validation (k-folds leave one out)
# auto generate 20 values between 1e-4 and 1e4 on log scale
clf = linear_model.LogisticRegressionCV(cv=5,
                                        Cs=20, ## takes awhile to fit 20 models!
                                        penalty='l1',
                                        solver='liblinear')
clf.fit(X_train,y_train)

LogisticRegressionCV(Cs=20, cv=5, penalty='l1', solver='liblinear')

In [40]:
## how many C's were fit?
clf.Cs
## which C's were fit?
clf.Cs_
## Which C was 'best'?
clf.C_

20

array([1.00000000e-04, 2.63665090e-04, 6.95192796e-04, 1.83298071e-03,
       4.83293024e-03, 1.27427499e-02, 3.35981829e-02, 8.85866790e-02,
       2.33572147e-01, 6.15848211e-01, 1.62377674e+00, 4.28133240e+00,
       1.12883789e+01, 2.97635144e+01, 7.84759970e+01, 2.06913808e+02,
       5.45559478e+02, 1.43844989e+03, 3.79269019e+03, 1.00000000e+04])

array([0.08858668])

In [41]:
## Scoring  the Model on Training and Testing Set
result_scores['Logistic_L1_C_auto'] = \
            (sklearn.metrics.accuracy_score(y_train,clf.predict(X_train)),
             sklearn.metrics.accuracy_score(y_test,clf.predict(X_test)))
get_results(result_scores)


Model                  Train    Test
-------------------------------------------
Logistic               0.7333   0.718 
Null                   0.6467   0.608 
Logistic_L1_C_1        0.732    0.716 
Logistic_L1_C_01       0.726    0.706 
Logistic_L1_C_10       0.7347   0.718 
Logistic_L1_C_auto     0.7233   0.708 


Scaling/pipeline

In [42]:
## LASSO (L1) regression, set alpha
from sklearn import preprocessing
from sklearn.pipeline import Pipeline
## set our transformation
scaler = preprocessing.StandardScaler()

## set our model
clf = linear_model.LogisticRegressionCV(cv=5,
                                        Cs=20, ## takes awhile to fit 20 models!
                                        penalty='l1',
                                        solver='liblinear')
## put together in pipeline
pipe1 = Pipeline([("scale", scaler),
                  ("LASSO", clf)])
pipe1.fit(X_train,y_train)

Pipeline(steps=[('scale', StandardScaler()),
                ('LASSO',
                 LogisticRegressionCV(Cs=20, cv=5, penalty='l1',
                                      solver='liblinear'))])

In [43]:
pipe1.named_steps

{'scale': StandardScaler(),
 'LASSO': LogisticRegressionCV(Cs=20, cv=5, penalty='l1', solver='liblinear')}

In [44]:
pipe1.named_steps['LASSO']

LogisticRegressionCV(Cs=20, cv=5, penalty='l1', solver='liblinear')

In [45]:
pipe1.named_steps['LASSO'].C_

array([0.08858668])

In [46]:
## Score the Model on Training and Testing Set
result_scores['Logistic_SL1_C_auto'] = \
            (sklearn.metrics.accuracy_score(y_train,pipe1.predict(X_train)),
             sklearn.metrics.accuracy_score(y_test,pipe1.predict(X_test)))
get_results(result_scores)


Model                  Train    Test
-------------------------------------------
Logistic               0.7333   0.718 
Null                   0.6467   0.608 
Logistic_L1_C_1        0.732    0.716 
Logistic_L1_C_01       0.726    0.706 
Logistic_L1_C_10       0.7347   0.718 
Logistic_L1_C_auto     0.7233   0.708 
Logistic_SL1_C_auto    0.7307   0.714 


Adding model complexity with interactions and polynomials

In [47]:
#### This takes awhile to run!
## try multiple polynomials with a LASSO regulaizer
## use pipeline for pre-processing
from sklearn.preprocessing import PolynomialFeatures

## set our transformation
scaler = preprocessing.StandardScaler()

## polynomials
poly_feat = PolynomialFeatures(degree=2,include_bias=False)

## set our model
clf = linear_model.LogisticRegressionCV(cv=3,
                                        Cs=5,
                                        penalty='l1',
                                        solver='liblinear')
## put together in pipeline
pipe2 = Pipeline([("scale", scaler),
                  ("poly",poly_feat),
                  ("LASSO", clf)])
# pipe2.fit(X_train,y_train)

Random Forest

In [49]:
#### Fit Random Forest
## Random Forests
from sklearn import ensemble
clf = ensemble.RandomForestClassifier(n_estimators=100,
                                      max_features=10,
                                      random_state=42)
clf.fit(X_train,y_train)
## get confusion matrix
confusion_matrix(y_train,clf.predict(X_train))

RandomForestClassifier(max_features=10, random_state=42)

array([[970,   0],
       [  1, 529]])

In [50]:
## Score the Model on Training and Testing Set
result_scores['RandomForest_noCV'] = \
            (sklearn.metrics.accuracy_score(y_train,clf.predict(X_train)),
             sklearn.metrics.accuracy_score(y_test,clf.predict(X_test)))
get_results(result_scores)


Model                  Train    Test
-------------------------------------------
Logistic               0.7333   0.718 
Null                   0.6467   0.608 
Logistic_L1_C_1        0.732    0.716 
Logistic_L1_C_01       0.726    0.706 
Logistic_L1_C_10       0.7347   0.718 
Logistic_L1_C_auto     0.7233   0.708 
Logistic_SL1_C_auto    0.7307   0.714 
RandomForest_noCV      0.9993   0.688 


Grid search for manual cross validation

In [51]:
from sklearn.model_selection import GridSearchCV
## specify grid
parameters = {'n_estimators':(50,100,200,300),
              'max_features':(5,10,15,20)}
## specify model without hyperparameters
rf_model = ensemble.RandomForestClassifier(random_state=32)
## specify search with model
clf = GridSearchCV(rf_model,
                   parameters,
                   cv=5,
                   return_train_score=True)
clf.fit(X_train,y_train)

GridSearchCV(cv=5, estimator=RandomForestClassifier(random_state=32),
             param_grid={'max_features': (5, 10, 15, 20),
                         'n_estimators': (50, 100, 200, 300)},
             return_train_score=True)

In [52]:
## exploring best hyperparameters
clf.best_params_

{'max_features': 15, 'n_estimators': 50}

In [53]:
## adding model score
## Scoring the Model on Training and Testing Set
result_scores['RandomForest_CV'] = \
            (sklearn.metrics.accuracy_score(y_train,clf.predict(X_train)),
             sklearn.metrics.accuracy_score(y_test,clf.predict(X_test)))
get_results(result_scores)


Model                  Train    Test
-------------------------------------------
Logistic               0.7333   0.718 
Null                   0.6467   0.608 
Logistic_L1_C_1        0.732    0.716 
Logistic_L1_C_01       0.726    0.706 
Logistic_L1_C_10       0.7347   0.718 
Logistic_L1_C_auto     0.7233   0.708 
Logistic_SL1_C_auto    0.7307   0.714 
RandomForest_noCV      0.9993   0.688 
RandomForest_CV        0.9953   0.692 


In [ ]:
clf.cv_results_

In [54]:
from sklearn.model_selection import GridSearchCV
## specify grid
parameters2 = {'max_depth':(2,5,7,10,20)}
## specify model without hyperparameters
rf_model = ensemble.RandomForestClassifier(max_features=20,
                                           n_estimators=100,
                                           random_state=32)
## specify search with model
clf = GridSearchCV(rf_model,
                   parameters2,
                   cv=5,
                   return_train_score=True)
clf.fit(X_train,y_train)

GridSearchCV(cv=5,
             estimator=RandomForestClassifier(max_features=20, random_state=32),
             param_grid={'max_depth': (2, 5, 7, 10, 20)},
             return_train_score=True)

In [55]:
## explore best hyperparameters
clf.best_params_

{'max_depth': 2}

In [56]:
## add model score
## Score the Model on Training and Testing Set
result_scores['RandomForest_CV2'] = \
            (sklearn.metrics.accuracy_score(y_train,clf.predict(X_train)),
             sklearn.metrics.accuracy_score(y_test,clf.predict(X_test)))
get_results(result_scores)


Model                  Train    Test
-------------------------------------------
Logistic               0.7333   0.718 
Null                   0.6467   0.608 
Logistic_L1_C_1        0.732    0.716 
Logistic_L1_C_01       0.726    0.706 
Logistic_L1_C_10       0.7347   0.718 
Logistic_L1_C_auto     0.7233   0.708 
Logistic_SL1_C_auto    0.7307   0.714 
RandomForest_noCV      0.9993   0.688 
RandomForest_CV        0.9953   0.692 
RandomForest_CV2       0.7273   0.706 


1.Among the different classification models included in the Python notebook, which model had the best overall performance? Support your response by referencing appropriate evidence.

Ans. From the above results we can say that Logistic Regression with L1 penalty and C=10

It has the highest train accuracy (0.7347) and ties for the highest test accuracy (0.718).

The plain Logistic model (no regularization) also scores 0.718 test accuracy, but Logistic_L1_C_10 has slightly better training accuracy, suggesting a tiny edge.

Other regularized logistic models have slightly lower performance.

Random Forest models had extremely high training accuracy, but noticeably lower test accuracy.

In [ ]:
Q2.

In [82]:
from sklearn.linear_model import LogisticRegression




In [83]:
print(X_train.dtypes)  # Check data types of features
print(y_train.dtypes)  # Check data types of target variable
print(X_train.isnull().sum())  # Check for missing values


PatientID                           int64
Gender                             object
Race                               object
Myocardial_infarction               int64
Congestive_heart_failure            int64
Peripheral_vascular_disease         int64
Stroke                              int64
Dementia                            int64
Pulmonary                           int64
Rheumatic                           int64
Peptic_ulcer_disease                int64
LiverMild                           int64
Diabetes_without_complications      int64
Diabetes_with_complications         int64
Paralysis                           int64
Renal                               int64
Cancer                              int64
LiverSevere                         int64
Metastatic_solid_tumour             int64
HIV                                 int64
Obesity                             int64
Depression                          int64
Hypertension                        int64
Drugs                             

In [84]:
X_train = X_train.apply(pd.to_numeric, errors='coerce')  # Convert columns to numeric
X_train = X_train.dropna()  # Drop rows with NaN values


Im using solvers like saga lbfgs and newton-cg. These have no regularization

In [93]:
import time
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder # Importing LabelEncoder

# Assuming df_patient is defined
X = df_patient.drop(columns=['mortality', 'DateOfBirth', 'DateOfDeath',
                              'First_Appointment_Date', 'Last_Appointment_Date',
                              'PatientID'])  # Excluding  datetime and ID columns
y = df_patient['mortality']

# Splitting  dataset into training and holdout subsets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Solvers list
solvers = ['saga', 'lbfgs', 'newton-cg']

# Initializing LabelEncoder
label_encoder = LabelEncoder()

# Converting  categorical columns to numerical using Label Encoding
for col in X_train.select_dtypes(include=['object']).columns:
    X_train[col] = label_encoder.fit_transform(X_train[col])
    X_test[col] = label_encoder.transform(X_test[col]) # Use the same encoder for X_test


results = []

for solver in solvers:
    # Timing the model training
    start_time = time.perf_counter()

    # Fitting Logistic Regression model without regularization
    # Changing 'none' to None
    model = LogisticRegression(penalty=None, solver=solver, max_iter=1000)
    model.fit(X_train, y_train)

    # Measuring time taken
    end_time = time.perf_counter()
    time_taken = end_time - start_time

    # Evaluating  accuracy
    train_accuracy = accuracy_score(y_train, model.predict(X_train))
    test_accuracy = accuracy_score(y_test, model.predict(X_test))

    # Storing  results and creating a table
    results.append({
        'Solver Used': solver,
        'Training Subset Accuracy': train_accuracy,
        'Holdout Subset Accuracy': test_accuracy,
        'Time Taken': time_taken
    })

# Displaying the results table
results_df = pd.DataFrame(results)
print(results_df)

LogisticRegression(max_iter=1000, penalty=None, solver='saga')

LogisticRegression(max_iter=1000, penalty=None)

LogisticRegression(max_iter=1000, penalty=None, solver='newton-cg')

  Solver Used  Training Subset Accuracy  Holdout Subset Accuracy  Time Taken
0        saga                  0.748500                   0.7360    4.933297
1       lbfgs                  0.748812                   0.7355    0.390781
2   newton-cg                  0.748500                   0.7360    0.141682


Q3. Compare the results of the models in terms of their accuracy (use this as the performance metric to assess generalizability error on the holdout subset) and the time taken (use appropriate timing function). Summarize your results via a table with the following structure:



In [ ]:
#
#SolverUsed	TrainingSubsetAccuracy	HoldoutSubsetAccuracy	 TimeTaken
#newton-cg    	0.7485	                0.7360                   0.1417
#lbfgs        	0.7488	                 0.7355	                 0.3908
#saga         	0.7485	                 0.7360	                 4.9333

From the results we can see that newton-cg is the fastest solver with 0.1417 seconds followed by lbfgs and saga is the slowest one with 4.9333. All of them have accuracy between 0.748

Q4. Based on the results, which solver yielded the best results? Explain the basis for ranking the models - did you use training subset accuracy? Holdout subset accuracy? Time of execution? All three? Some combination of the three?

In this analysis, the holdout subset accuracy is the important factor , as it indicates how well the model generalizes to unseen data. Both newton-cg and saga achieved identical accuracy (0.7360), while lbfgs was slightly lower (0.7355). Training subset accuracy was similar across all solvers,having minimal difference. The key difference is execution time, where newton-cg was the fastest (0.1417 seconds), followed by lbfgs (0.3908 seconds), and saga was the slowest (4.93 seconds). If speed is essential, newton-cg is ideal, but lbfgs offers a good balance of speed and accuracy.